In [3]:
import pandas as pd 
import numpy as np 
import os

In [9]:
def data_generator():
    
    directories = os.listdir('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')
    directories = directories[:1]#temporal, indica cuantos batches se van a tomar
    directories = [('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')+x for i, x in enumerate(directories)]
    
    data = pd.DataFrame()
    
    
    for file in directories:
        temp = pd.read_parquet(file)
        
        temp = temp.sort_values('charge',ascending = False).groupby('event_id').head(2)
        # al poner el head en 100 se reduciria la tabla en un 98%
    
        data = pd.concat([data, temp], axis=0)
        
    
    #data = data.drop_duplicates()##
    
    coords = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
    train_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet')
    
    data['event'] = data.index
    
    
    data = pd.merge(data, train_meta[['event_id', 'azimuth', 'zenith']], left_on=data['event'], right_on=train_meta['event_id'].astype(float))
    
    data = pd.merge(data, coords, on='sensor_id', how='outer')


     
    data = pd.concat([
        data.groupby('event').agg({'time': lambda x: list(x)}),
        data.groupby('event').agg({'charge': lambda x: list(x)}),
        data.groupby('event').agg({'x': lambda x: list(x)}),
        data.groupby('event').agg({'y': lambda x: list(x)}),
        data.groupby('event').agg({'z': lambda x: list(x)}),
        data.groupby('event')['azimuth'].apply(min),
        data.groupby('event')['zenith'].apply(min)
        ],axis=1)
    
    return data[['time','charge','x','y','z','azimuth','zenith']]



In [10]:
data = data_generator()

In [23]:
data.head(4)

,time,charge,x,y,z,azimuth,zenith
event,,,,,,,
777938857.0,"[11127.0, 10436.0]","[2.2750000953674316, 2.174999952316284]","[-32.96, 303.41]","[62.44, 335.64]","[19.23, 70.42]",0.095967,2.747172
777938862.0,"[10404.0, 10855.0]","[2.674999952316284, 1.875]","[11.87, 505.27]","[179.19, 257.88]","[-365.71, -140.55]",2.533461,0.638181
777938866.0,"[11231.0, 11027.0]","[2.075000047683716, 1.975000023841858]","[-200.55, -166.4]","[-74.03, -287.79]","[-225.49, -421.34]",2.510193,1.059650
777938893.0,"[15461.0, 6528.0]","[1.774999976158142, 1.774999976158142]","[-200.55, 237.78]","[-74.03, -442.42]","[-208.47, 382.32]",4.844315,1.450211


In [6]:
import tensorflow as tf 
import keras

In [11]:
size = 0.25
epoch = 30

test_size = int(round((len(data)*size),0))
data.sample(frac=1)


X = data[['time','charge','x','y','z']]
y_az = data['azimuth']
y_ze = data['zenith']


num_cols = len(X.columns)
num_rows = len(X)
res = list(X.sum(axis=1).to_numpy())
res = np.reshape(res, (num_rows, num_cols, -1))
X = res


X_train = X[test_size:]
X_test = X[:test_size]

y_az_train = y_az.iloc[test_size:]
y_az_test = y_az.iloc[:test_size]

y_ze_train = y_ze.iloc[test_size:]
y_ze_test = y_ze.iloc[:test_size]


In [ ]:
input = keras.Input(shape=(5, 100, ))

x = keras.layers.Flatten()(input)

xa = keras.layers.Dense(128, activation='linear')(x)
xa = keras.layers.Dense(64, activation='linear')(xa)
xa = keras.layers.Dense(32, activation='linear')(xa)

xz = keras.layers.Dense(128, activation='linear')(x)
xz = keras.layers.Dense(64, activation='linear')(xz)
xz = keras.layers.Dense(32, activation='linear')(xz)

out_az = keras.layers.Dense(1, activation='relu', name='az-out')(xa)
out_ze = keras.layers.Dense(1, activation='relu', name='ze-out')(xz)


model = keras.Model( inputs = input, outputs = [out_az, out_ze])


model.compile(
    loss = {
        'az-out': tf.keras.losses.MeanAbsoluteError(),
        'ze-out': tf.keras.losses.MeanAbsoluteError(),
    },
    metrics = {
        'az-out': 'accuracy',
        'ze-out': 'accuracy',
    },
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
    
        
    
history = model.fit(x=X_train,
            y={
                'az-out': y_az_train,
                'ze-out': y_ze_train
            },
            validation_data=(X_test,             
            {
                'az-out': y_az_test,
                'ze-out': y_ze_test
            }),
              epochs=epoch,verbose=1)
    

In [26]:
tf.convert_to_tensor(c)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
numpy.array([numpy.array(xi) for xi in x])

In [32]:
d = np.zeros((2, 3, 4))
d.shape

(2, 3, 4)

In [88]:
pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/sample_submission.parquet').head()

,event_id,azimuth,zenith
0,2092,1,1
1,7344,1,1
2,9482,1,1


In [ ]:
def data_test():
    
    directories = os.listdir('/kaggle/input/icecube-neutrinos-in-deep-ice/test/')
    directories = directories[:2]#temporal, indica cuantos batches se van a tomar
    directories = [('/kaggle/input/icecube-neutrinos-in-deep-ice/test/')+x for i, x in enumerate(directories)]
    
    data = pd.DataFrame()
    
    
    for file in directories:
        temp = pd.read_parquet(file)
        
        temp = temp.sort_values('charge',ascending = False).groupby('event_id').head(2)
        # al poner el head en 100 se reduciria la tabla en un 98%
    
        data = pd.concat([data, temp], axis=0)
        
    
    #data = data.drop_duplicates()##
    
    coords = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
    
    data['event_id'] = data.index
    
    data = pd.merge(data, coords, on='sensor_id', how='outer')

     
    data = pd.concat([
        data.groupby('event').agg({'event_id': lambda x: min(x)}),
        data.groupby('event').agg({'time': lambda x: list(x)}),
        data.groupby('event').agg({'charge': lambda x: list(x)}),
        data.groupby('event').agg({'x': lambda x: list(x)}),
        data.groupby('event').agg({'y': lambda x: list(x)}),
        data.groupby('event').agg({'z': lambda x: list(x)})
        ],axis=1)
    
    
    return data[['event_id','time','charge','x','y','z']]

In [ ]:
test_data = data_test()

In [ ]:
test_data[['azimuth', 'zenith']] = model.predict(test_data[['time', 'charge', 'x', 'y', 'z']])

In [ ]:
test_data[['event_id','azimuth','zenith']].to_parquet('submission.parquet')